# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
!pip install chromadb ollama langchain_huggingface sentence-transformers google-generativeai --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.9.2 requires tokenizers>=0.21.1, but you have tokenizers 0.20.3 which is incompatible.
vllm 0.9.2 requires transformers>=4.51.1, but you have transformers 4.46.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew,LLM

In [4]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"


groq_llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

In [5]:
llmconfig = dict(
    llm=dict(
        provider="openai",
        config=dict(
            model="ibm-granite/granite-3.3-2b-instruct",
            base_url=f"{INFERENCE_SERVER_URL}/v1",
            api_key=API_KEY,
        ),
    ),
    embedder=dict(
        provider="huggingface",#,.goolge
        config=dict(
            model="BAAI/bge-small-en-v1.5"#"models/embedding-001"#"nomic-ai/nomic-embed-text-v1"
        ),
    ),
)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [7]:
# Define file paths for YAML configurations
files = {
    'agents': 'L5-config/agents.yaml',
    'tasks': 'L5-config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

website_search_tool = WebsiteSearchTool(
config=dict(
        llm=dict(
            provider="openai", # Options include ollama, google, anthropic, llama2, and more
            config=dict(
                model="ibm-granite/granite-3.3-2b-instruct",
                base_url=f"{INFERENCE_SERVER_URL}/v1",
                api_key=API_KEY,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
            ),
        ),
    )
)

/opt/app-root/lib64/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/opt/app-root/lib64/python3.11/site-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
/opt/app-root/lib64/python3.11/site-packages/embedchain/embedder/huggingface.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `

## Setup Multi LLM models

In [9]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
# groq_llm = "groq/llama-3.1-70b-versatile"
os.environ['SERPER_API_KEY'] ="30bdec95f601d48ca27880feedef448ca840b2f0"

## Creating Crew, Agents, and Tasks

In [10]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), website_search_tool],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), website_search_tool],
    llm=groq_llm,
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=groq_llm,
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [11]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d85ef51b-46f9-45dc-8b9f-ea3cd23603e2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Market Analyst                                                                                     │
│                                                                                                                 │
│  Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular      │
│  focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most       │
│  relevant and impactful news items that could influence market trends or investor decisions. Utilize financial  │
│  news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends,         │
│  regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the     │
│  impact on the stock market in 2024.                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Lead Market Analyst                                                                                     │
│                                                                                                                 │
│  Thought: Thought: I need to use the "Search the internet with Serper" tool to find the most relevant           │
│  financial news articles focusing on the US inflation in 2024 and its impact on the stock market. I will also   │
│  read specific websites that offer comprehensive market analysis and economic predictions for accuracy.         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "article": "# US Inflation and Stock Market Outlook for 2024\n\n## Introduction\n\nAs we transition into     │
│  2024, the dynamics between US inflation and stock market performance gain critical attention, with analysts    │
│  forecasting varied trajectories to guide market movements. This article delves into the projections,           │
│  identifying patterns, prospects, and risk factors.\n\n## Moderately Moderating Inflation: The Base Case        │
│  Scenario\n\nThe baseline forecast for 2024 positions inflation at 2.9%, with tariffs slightly mitigating       │
│  their impact towards year-end. The Federal Reserve is poised to execute subtle policy rate reductions, while   │
│  GDP growth is projected to edge up to 1.4% annually through 2026.\n\n### Consumer Spending Dynamics\n\n- Real  │
│  consumer spending is anticipated to rise at a gentle 1.4%, constrained by elevated interest rates and          │
│  tariffs.\n\n  *Opportunity:* Investors can explore strategies targeting sectors such as durable goods (large   │
│  appliances, furniture) as consumers become more cautious with their expenditures. Continued consumer           │
│  monitoring will be key.\n\n### Business Investment Trends\n\n- Investment growth forecasts to slow in 2025     │
│  and 2026, influenced by weakened consumer confidence and elevated borrowing costs.\n\n  *Insight:* Content     │
│  can highlight resilient firms in challenging conditions by presenting case studies and sharing insightful      │
│  expert opinions.\n\n- Domestic equipment investment is projected to bounce back in Q2 2026 post-temporary      │
│  dips in 2025 due to tariffs. Conversely, machinery and equipment investments are predicted to decline by 2.5%  │
│  in 2026, although intellectual property investments, less influenced by tariffs, are expected to rise by       │
│  3.7%.\n\n### Foreign Trade and Policy Tensions\n\n- Trade policies' uncertainties slightly alter               │
│  international supply chain decisions, leading to an overall decline in exports and imports by year-end, a      │
│  7.1% decrease in 2026 and a 1.8% reduction, respectively. This results in a combined 9.2% retraction.\n\n      │
│  *Content Framework:* Engage readers by creating articles presenting contrasting trade policy viewpoints or     │
│  featuring balanced expert opinions.\n\n### Labor Markets and Inflation\n\n- Projections suggest an upward      │
│  movement to 4.6% unemployment by 2026, with the core CPI averaging 2.9% in both 2025 and 2026, citing minimal  │
│  tariff influence on retail costs.\n\n  *Engagement Strategy:* Design an eye-catching infographic contrasting   │
│  inflation levels with base case projections and vital inflection points.\n\n### Stocks Market Outlook\n\n-     │
│  The S&P 500 is expected to initially slip in 2025 amid demand erosion, rising rates, and tariff concerns, yet  │
│  rebounds in Q2 and stabilize organically post-year-end, benefitting from stable tariff policy and bolstered    │
│  market clarity.\n\n## Intensified Inflation Downside Scenario\n\nFor a more adverse outlook - significant      │
│  tariff escalations (25% by year-end) bundled with elevated interest rates, could trigger recessionary          │
│  conditions commencing Q4 2025.\n\n### Adverse Consumer

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fbafadb5-5fda-49b5-a7d7-db85ead89f56                                                                     │
│  Agent: Chief Content Officer                                                                                   │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d85ef51b-46f9-45dc-8b9f-ea3cd23603e2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "article": "# US Inflation and Stock Market Outlook for 2024\n\n## Introduction\n\nAs we transition into     │
│  2024, the dynamics between US inflation and stock market performance gain critical attention, with analysts    │
│  forecasting varied trajectories to guide market movements. This article delves into the projections,           │
│  identifying patterns, prospects, and risk factors.\n\n## Moderately Moderating Inflation: The Base Case        │
│  Scenario\n\nThe baseline forecast for 2024 positions inflation at 2.9%, with tariffs slightly mitigating       │
│  their impact towards year-end. The Federal Reserve is poised to execute subtle policy rate reductions, while   │
│  GDP growth is projected to edge up to 1.4% annually through 2026.\n\n### Consumer Spending Dynamics\n\n- Real  │
│  consumer spending is anticipated to rise at a gentle 1.4%, constrained by elevated interest rates and          │
│  tariffs.\n\n  *Opportunity:* Investors can explore strategies targeting sectors such as durable goods (large   │
│  appliances, furniture) as consumers become more cautious with their expenditures. Continued consumer           │
│  monitoring will be key.\n\n### Business Investment Trends\n\n- Investment growth forecasts to slow in 2025     │
│  and 2026, influenced by weakened consumer confidence and elevated borrowing costs.\n\n  *Insight:* Content     │
│  can highlight resilient firms in challenging conditions by presenting case studies and sharing insightful      │
│  expert opinions.\n\n- Domestic equipment investment is projected to bounce back in Q2 2026 post-temporary      │
│  dips in 2025 due to tariffs. Conversely, machinery and equipment investments are predicted to decline by 2.5%  │
│  in 2026, although intellectual property investments, less influenced by tariffs, are expected to rise by       │
│  3.7%.\n\n### Foreign Trade and Policy Tensions\n\n- Trade policies' uncertainties slightly alter               │
│  international supply chain decisions, leading to an overall decline in exports and imports by year-end, a      │
│  7.1% decrease in 2026 and a 1.8% reduction, respectively. This results in a combined 9.2% retraction.\n\n      │
│  *Content Framework:* Engage readers by creating articles presenting contrasting trade policy viewpoints or     │
│  featuring balanced expert opinions.\n\n### Labor Markets and Inflation\n\n- Projections suggest an upward      │
│  movement to 4.6% unemployment by 2026, with the core CPI averaging 2.9% in both 2025 and 2026, citing minimal  │
│  tariff influence on retail costs.\n\n  *Engagement Strategy:* Design an eye-catching infographic contrasting   │
│  inflation levels with base case projections and vital inflection points.\n\n### Stocks Market Outlook\n\n-     │
│  The S&P 500 is expected to initially slip in 2025 amid demand erosion, rising rates, and tariff concerns, yet  │
│  rebounds in Q2 and stabilize organically post-year-end, benefitting from stable tariff policy and bolstered    │
│  market clarity.\n\n## Intensified Inflation Downside Scenario\n\nFor a more adverse outlook - significant      │
│  tariff escalations (25% by year-end) bundled with ele

## Social Content

In [12]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
TMY: Navigating the US Inflation-Stock Market
Interplay for 2024. Moderate Inflation Scenario
vs. Riskier Downside Projections. Read Now 🔗 [link
to article]
--------------------------------------------------
LinkedIn
Exploring the Moderately Moderating Inflation
Outlook for 2024 and Its Potential Stock Market
Impact Insights on Trends by TMY  - Discover the
projected 2.9% inflation rate and 1.4% GDP growth
in this detailed outlook.   Read More: [link to
article]
--------------------------------------------------


/tmp/ipykernel_2940/3812343593.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  posts = result.pydantic.dict()['social_media_posts']


## Blog Post

In [13]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

/tmp/ipykernel_2940/3784749904.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  display(Markdown(result.pydantic.dict()['article']))


TMY: Navigating the US Inflation-Stock Market Interplay for 2024. Moderate Inflation Scenario vs. Riskier Downside Projections. Read Now 🔗 [link to article]